In [1]:
import pandas as pd
import os
from tqdm.notebook import tqdm
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import texttable
import latextable
import json

tags = {
    "loss": "Loss_MB/train_phase/train_stream/Task000",
    "final_accuracy": "Accuracy_On_Trained_Experiences/eval_phase/test_stream/Task000",
}

def load_event_file(experiment_directory: str) -> EventAccumulator:
    """
    Loads the event file from a given experiment directory.
    """
    ea: EventAccumulator = EventAccumulator(experiment_directory)
    ea.Reload()
    return ea

def final_metrics(ea: EventAccumulator) -> dict:
    """
    Returns the final metrics from the event file.
    """
    result = {}
    for tag_name, tag in tags.items():
        if tag in ea.Tags()['scalars']:
            result[tag_name] = ea.Scalars(tag)[-1].value
    return result



In [4]:
experiment_logs = "experiment_logs"

records = []

# Loop over each directory in a directory
for experiment in tqdm(os.listdir(os.path.join(experiment_logs))):
    experiment_code = experiment.split("_")
    i, host, repo_hash, experiment_category, dataset, arch, strategy = experiment_code


    record = {
        "dataset": dataset,
        "architecture": arch,
        "strategy": strategy,
        "experiment_category": experiment_category,
        "id": i,
        "repo_hash": repo_hash
    }

    try:
        ea = load_event_file(os.path.join(experiment_logs, experiment))
        record.update(final_metrics(ea))
    except KeyError as e:
        print(f"Could not load required metric {e} in '{experiment}'")

    try:
        with open(os.path.join(experiment_logs, experiment, "config.json"), "r") as f:
            config = json.load(f)
            record.update(config)
    except FileNotFoundError as e:
        print(f"Could not load config file '{experiment}/config.json'")

    records.append(record)

df = pd.DataFrame.from_dict(records)
df

  0%|          | 0/184 [00:00<?, ?it/s]

Could not load config file '0083_ml-24_dd88ddf_N_splitEmbeddedCIFAR100_AE_taskInference/config.json'
Could not load config file '0204_ml-22_a7f3111e_OS_splitFMNIST_AE_strategy/config.json'
Could not load config file '0081_ml-24_dd88ddf_N_splitEmbeddedCIFAR100_AE_taskInference/config.json'
Could not load config file '0082_ml-24_dd88ddf_N_splitEmbeddedCIFAR100_AE_taskInference/config.json'
Could not load config file '0084_ml-24_dd88ddf_N_splitEmbeddedCIFAR100_AE_taskInference/config.json'
Could not load config file '0004_ml-24_dd88ddf_N_splitFMNIST_VAE_taskOracle/config.json'
Could not load config file '0003_ml-24_dd88ddf_N_splitFMNIST_AE_taskOracle/config.json'
Could not load config file '0006_ml-24_dd88ddf_N_splitFMNIST_VAE_taskInference/config.json'
Could not load config file '0005_ml-24_dd88ddf_N_splitFMNIST_AE_taskInference/config.json'
Could not load config file '0203_ml-22_a7f3111e_OS_splitFMNIST_AE_strategy/config.json'
Could not load config file '0101_ml-24_dd88ddf_N_splitEmbedd

,dataset,architecture,strategy,experiment_category,id,repo_hash,loss,final_accuracy,batch_size,classifier_loss_weight,...,task_inference_strategy,vanilla_cnn_config,use_experience_replay,use_learning_without_forgetting,use_synaptic_intelligence,rectangular_mlp_config,lwf_alpha,replay_buffer,si_lambda,use_generative_replay_strategy
0,splitEmbeddedCIFAR100,VAE,cumulative,N,0058,dd88ddf,1.014315,0.313800,64.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,splitEmbeddedCIFAR100,AE,taskInference,N,0083,dd88ddf,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,splitCORe50,VAE,taskInference,N,0046,dd88ddf,1.803232,0.053100,64.0,1.0,...,task_reconstruction_loss,{'base_channels': 128},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,splitCORe50,VAE,finetuning,N,0030,dd88ddf,0.239605,0.075069,64.0,1.0,...,NaN,{'base_channels': 128},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,splitCIFAR100,VAE,taskInference,EP,0013,2ac2112a,NaN,0.011800,64.0,1.0,...,task_reconstruction_loss,NaN,False,False,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,splitEmbeddedCORe50,VAE,taskInference,N,0122,dd88ddf,NaN,0.375611,64.0,1.0,...,task_reconstruction_loss,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,splitEmbeddedCIFAR100,AE,taskInference,N,0095,dd88ddf,2.274977,0.346600,64.0,1.0,...,task_reconstruction_loss,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,splitCIFAR10,AE,taskOracle,EP,0002,b7711e62,NaN,0.927700,64.0,1.0,...,task_oracle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,splitEmbeddedCIFAR100,AE,taskInference,N,0071,dd88ddf,0.835625,0.161500,64.0,1.0,...,task_reconstruction_loss,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.to_csv("results/results.csv")

In [6]:
# 
# Regular Experiment
# 
for strategy in ["cumulative", "taskOracle", "finetuning", "taskInference"]:
    for arch in ["AE", "VAE"]:
        for dataset in [
            "splitFMNIST",
            "splitCIFAR10",
            "splitCIFAR100",
            "splitCORe50",
            "splitEmbeddedCIFAR100",
            "splitEmbeddedCORe50"]:
            try:
                result = df[df['dataset'] == dataset]
                result = result[result['prune_strategy'] == "EXP"]
                result = result[result['strategy'] == strategy]
                result = result[result['architecture'] == arch]
                accuracy = result.sort_values(by='id').iloc[-1]["final_accuracy"]
                # accuracy = result.sort_values("id")[-1]["accuracy"]
                # display(result.sort_values(by='id'))
                print(f"{accuracy:0.4f}", end="\t")
            except:
                continue
        print()

In [21]:
def bold_column_max(tt: texttable.Texttable) -> texttable.Texttable:
    """
    Bold the maximum value in each column. Ignores text in the column.
    """
    column_maxes = [0] * len(tt._rows[0])

    for row in tt._rows:
        for i, cell in enumerate(row):
            try:
                column_maxes[i] = max(column_maxes[i], float(cell))
            except:
                pass

    for row in tt._rows:
        for i, cell in enumerate(row):
            try:
                numeric_cell = float(""+cell)
                if column_maxes[i] == numeric_cell:
                    row[i] = "\\textbf{"+cell+"}"
            except:
                pass
    return tt

In [23]:
# 
# Equal Prune Experiment
# 
def table_equal_prune(experiment_results: pd.DataFrame):
    df = experiment_results.copy()

    equal_prune_table = texttable.Texttable()
    columns = ["splitFMNIST", "splitCIFAR10", "splitCIFAR100", "splitCORe50", "splitEmbeddedCIFAR100", "splitEmbeddedCORe50"]
    
    equal_prune_table.header(["", ""] + columns)

    for strategy in ["taskOracle", "taskInference"]:
        for arch in ["AE", "VAE"]:
            row = [strategy, arch]
            for dataset in columns:
                try:
                    result = df.loc[
                        (df['dataset'] == dataset) &
                        (df['strategy'] == strategy) &
                        (df['architecture'] == arch)
                    ]
                    accuracy = result.sort_values(by='id').iloc[-1]["final_accuracy"]
                    row.append(f"{accuracy*100:0.1f}\%")
                except:
                    row.append(f"NA")
            equal_prune_table.add_row(row)

    print(equal_prune_table.draw())

    print(latextable.draw_latex(
        equal_prune_table,
        caption="Equal Prune Experiment",
        label="tab:equal_prune_experiment",
        use_booktabs=True
    ))

table_equal_prune(df[df["experiment_category"]=="EP"])

+----------+-----+----------+----------+---------+---------+---------+---------+
|          |     | splitFMN | splitCIF | splitCI | splitCO | splitEm | splitEm |
|          |     |   IST    |   AR10   | FAR100  |  Re50   | beddedC | beddedC |
|          |     |          |          |         |         | IFAR100 |  ORe50  |
+==========+=====+==========+==========+=========+=========+=========+=========+
| taskOrac | AE  | 99.3\%   | 93.4\%   | 55.7\%  | 50.7\%  | 56.1\%  | 75.9\%  |
| le       |     |          |          |         |         |         |         |
+----------+-----+----------+----------+---------+---------+---------+---------+
| taskOrac | VAE | nan\%    | 91.7\%   | 53.0\%  | 54.1\%  | 7.1\%   | 10.2\%  |
| le       |     |          |          |         |         |         |         |
+----------+-----+----------+----------+---------+---------+---------+---------+
| taskInfe | AE  | 99.3\%   | 43.0\%   | 25.7\%  | 8.2\%   | NA      | NA      |
| rence    |     |          

In [ ]:
df[(df["experiment_category"]=="EP") & (df["dataset"]=="splitEmbeddedCIFAR100")) ]

,dataset,architecture,strategy,experiment_category,id,repo_hash,loss,final_accuracy,batch_size,classifier_loss_weight,...,retrain_epochs,total_task_epochs,use_classifier_loss,use_packnet,use_reconstruction_loss,use_vae_loss,vae_loss_weight,task_inference_strategy,vanilla_cnn_config,rectangular_mlp_config
5,splitCIFAR10,AE,taskInference,EP,0143,dd88ddf,NaN,0.430300,64.0,1.0,...,10.0,50.0,True,True,True,False,0.001,task_reconstruction_loss,NaN,NaN
7,splitFMNIST,VAE,taskInference,EP,0146,dd88ddf,NaN,0.993000,64.0,1.0,...,5.0,20.0,True,True,True,True,0.001,task_reconstruction_loss,NaN,NaN
14,splitCORe50,AE,taskOracle,EP,0004,b7711e62,NaN,0.366917,64.0,1.0,...,1.0,5.0,True,True,True,False,NaN,task_oracle,NaN,NaN
16,splitCIFAR10,VAE,taskOracle,EP,0139,dd88ddf,NaN,0.917200,64.0,1.0,...,10.0,50.0,True,True,True,True,0.001,task_oracle,NaN,NaN
19,splitFMNIST,VAE,taskInference,EP,0013,4f911459,NaN,NaN,64.0,1.0,...,5.0,20.0,True,True,True,True,1.000,task_reconstruction_loss,NaN,NaN
23,splitCORe50,VAE,taskOracle,EP,0141,dd88ddf,NaN,0.540648,64.0,1.0,...,10.0,50.0,True,True,True,True,0.001,task_oracle,NaN,NaN
32,splitCIFAR100,AE,taskOracle,EP,0136,dd88ddf,NaN,0.556700,64.0,1.0,...,20.0,100.0,True,True,True,False,NaN,task_oracle,NaN,NaN
33,splitCORe50,AE,taskInference,EP,0145,dd88ddf,NaN,0.082474,64.0,1.0,...,10.0,50.0,True,True,True,False,0.001,task_reconstruction_loss,NaN,NaN
71,splitCIFAR10,VAE,taskOracle,EP,0008,4f911459,NaN,0.566200,64.0,1.0,...,10.0,50.0,True,True,True,True,1.000,task_oracle,NaN,NaN
75,splitFMNIST,VAE,taskOracle,EP,0007,4f911459,NaN,0.593100,64.0,1.0,...,5.0,20.0,True,True,True,True,1.000,task_oracle,NaN,NaN


In [ ]:
# 
# Equal Prune Experiment
# 
def table_prune_levels(experiment_results: pd.DataFrame):
    df = experiment_results.copy()

    equal_prune_table = texttable.Texttable()
    columns = ["splitFMNIST", "splitCIFAR10", "splitCIFAR100", "splitCORe50", "splitEmbeddedCIFAR100", "splitEmbeddedCORe50"]
    
    equal_prune_table.header(["Prune Level"] + columns)

    for prune_level in [0.2, 0.4, 0.5, 0.6, 0.8]:
            row = [f"{prune_level*100:0.0f}\%"]
            for dataset in columns:
                try:
                    result = df.loc[
                        (df['dataset'] == dataset) &
                        (df['prune_proportion'] == prune_level) &
                        (df['architecture'] == "AE")
                    ]
                    accuracy = result.sort_values(by='id').iloc[-1]["final_accuracy"]
                    row.append(f"{accuracy*100:0.1f}\%")
                except:
                    row.append(f"NA")
            equal_prune_table.add_row(row)

    print(equal_prune_table.draw())

    print(latextable.draw_latex(
        equal_prune_table,
        caption="Multiple Prune Levels with Task Inference AutoEncoder",
        label="tab:prune_levels",
        use_booktabs=True
    ))

table_prune_levels(df[
    (df["experiment_category"]=="PL") & 
    ((df["repo_hash"] == "55dff8e1") | (df["repo_hash"] == "7ee898bd"))
])


+-----------+-----------+----------+----------+----------+----------+----------+
|   Prune   | splitFMNI | splitCIF | splitCIF | splitCOR | splitEmb | splitEmb |
|   Level   |    ST     |   AR10   |  AR100   |   e50    | eddedCIF | eddedCOR |
|           |           |          |          |          |  AR100   |   e50    |
+===========+===========+==========+==========+==========+==========+==========+
| 20\%      | 73.6\%    | 36.7\%   | 17.1\%   | 6.7\%    | 17.4\%   | 21.7\%   |
+-----------+-----------+----------+----------+----------+----------+----------+
| 40\%      | 99.4\%    | 43.5\%   | 21.5\%   | 5.9\%    | 33.5\%   | 35.1\%   |
+-----------+-----------+----------+----------+----------+----------+----------+
| 50\%      | 99.3\%    | 56.3\%   | 28.7\%   | 6.4\%    | 40.6\%   | 44.1\%   |
+-----------+-----------+----------+----------+----------+----------+----------+
| 60\%      | 99.4\%    | 78.6\%   | 17.3\%   | 9.0\%    | 51.1\%   | 54.8\%   |
+-----------+-----------+---